In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
from datetime import datetime
import os

def get_current_season():
    """Determine current season based on date"""
    now = datetime.now()
    year = now.year
    month = now.month
    
    if month >= 8:
        return str(year)
    else:
        return str(year - 1)

# Configuration
leagues = {
    'EPL': 'EPL',
    'La_Liga': 'La_liga',
    'Bundesliga': 'Bundesliga', 
    'Serie_A': 'Serie_A',
    'Ligue_1': 'Ligue_1',
    'RFPL': 'RFPL'
}

def scrape_team_season(league, season):
    """Scrape team statistics for a specific league and season"""
    url = f'https://understat.com/league/{league}/{season}'
    
    try:
        res = requests.get(url, timeout=30)
        res.raise_for_status()
        soup = BeautifulSoup(res.content, 'lxml')
        
        # Find the script containing teamsData
        scripts = soup.find_all('script')
        json_string = ''
        
        for script in scripts:
            if 'teamsData' in str(script):
                json_string = str(script).strip()
                break
        
        if not json_string:
            return None
            
        # Extract JSON data
        start = json_string.index("('") + 2
        end = json_string.index("')")
        json_data = json_string[start:end]
        json_data = json_data.encode('utf8').decode('unicode_escape')
        data = json.loads(json_data)
        
        # Process each team
        all_teams = []
        for team_id, team_data in data.items():
            team_stats = {
                'team_id': team_id,
                'team_name': team_data['title'],
                'league': league,
                'year': int(season),
                'season': f"{season}/{str(int(season)+1)[2:]}"
            }
            
            # Get season totals from history
            if 'history' in team_data and team_data['history']:
                history = team_data['history']
                
                # Initialize counters
                totals = {
                    'matches': len(history),
                    'wins': 0,
                    'draws': 0,
                    'losses': 0,
                    'scored': 0,
                    'conceded': 0,
                    'xG': 0,
                    'xGA': 0,
                    'npxG': 0,
                    'npxGA': 0,
                    'deep': 0,
                    'deep_allowed': 0,
                    'pts': 0,
                    'xpts': 0
                }
                
                ppda_att = []
                ppda_def = []
                oppda_att = []
                oppda_def = []
                
                # Aggregate match data
                for match in history:
                    # Count results
                    if match['result'] == 'w':
                        totals['wins'] += 1
                    elif match['result'] == 'd':
                        totals['draws'] += 1
                    else:
                        totals['losses'] += 1
                    
                    # Sum statistics
                    totals['scored'] += int(match.get('scored', 0))
                    totals['conceded'] += int(match.get('missed', 0))
                    totals['xG'] += float(match.get('xG', 0))
                    totals['xGA'] += float(match.get('xGA', 0))
                    totals['npxG'] += float(match.get('npxG', 0))
                    totals['npxGA'] += float(match.get('npxGA', 0))
                    totals['deep'] += int(match.get('deep', 0))
                    totals['deep_allowed'] += int(match.get('deep_allowed', 0))
                    totals['pts'] += int(match.get('pts', 0))
                    totals['xpts'] += float(match.get('xpts', 0))
                    
                    # PPDA calculations
                    if 'ppda' in match and match['ppda']:
                        ppda_att.append(match['ppda'].get('att', 0))
                        ppda_def.append(match['ppda'].get('def', 1))
                    
                    if 'ppda_allowed' in match and match['ppda_allowed']:
                        oppda_att.append(match['ppda_allowed'].get('att', 0))
                        oppda_def.append(match['ppda_allowed'].get('def', 1))
                
                # Calculate PPDA coefficients
                if ppda_att and ppda_def:
                    totals['ppda_coef'] = sum(ppda_att) / sum(ppda_def) if sum(ppda_def) > 0 else 0
                else:
                    totals['ppda_coef'] = 0
                    
                if oppda_att and oppda_def:
                    totals['oppda_coef'] = sum(oppda_att) / sum(oppda_def) if sum(oppda_def) > 0 else 0
                else:
                    totals['oppda_coef'] = 0
                
                # Add calculated fields
                totals['npxGD'] = totals['npxG'] - totals['npxGA']
                totals['xG_diff'] = totals['xG'] - totals['scored']
                totals['xGA_diff'] = totals['xGA'] - totals['conceded']
                totals['xpts_diff'] = totals['xpts'] - totals['pts']
                
                # Merge with team info
                team_stats.update(totals)
                all_teams.append(team_stats)
        
        return pd.DataFrame(all_teams)
        
    except Exception as e:
        print(f"Error scraping {league} {season}: {str(e)}")
        return None

def scrape_all_team_data():
    """Scrape team data for all leagues and seasons"""
    
    seasons = ['2014', '2015', '2016', '2017', '2018', '2019', 
               '2020', '2021', '2022', '2023', '2024']
    current_season = get_current_season()
    
    print("⚽ UNDERSTAT TEAM DATA SCRAPER")
    print("=" * 60)
    
    # Historical data (2014-2024)
    historical_data = []
    
    print("\n📚 Scraping historical data (2014-2024)...")
    for season in seasons:
        print(f"\n Season {season}:")
        for league_name, league_code in leagues.items():
            print(f"  {league_name}...", end=" ")
            
            df = scrape_team_season(league_code, season)
            if df is not None and not df.empty:
                historical_data.append(df)
                print(f"✅ {len(df)} teams")
            else:
                print("⚠️ No data")
            
            time.sleep(1.5)
    
    # Save historical data
    if historical_data:
        historical_df = pd.concat(historical_data, ignore_index=True)
        historical_df = historical_df.sort_values(['year', 'league', 'pts'], ascending=[True, True, False])
        historical_df['position'] = historical_df.groupby(['year', 'league']).cumcount() + 1
        historical_df['scrape_timestamp'] = datetime.now().isoformat()
        
        # Save files
        historical_df.to_csv('understat_teams_aggregated_2014_2024.csv', index=False)
        historical_df.to_parquet('understat_teams_aggregated_2014_2024.parquet', index=False)
        print(f"\n✅ Historical data saved: {len(historical_df)} team-seasons")
    
    # Current season data
    print(f"\n📊 Scraping current season ({current_season})...")
    current_data = []
    
    for league_name, league_code in leagues.items():
        print(f"  {league_name}...", end=" ")
        
        df = scrape_team_season(league_code, current_season)
        if df is not None and not df.empty:
            current_data.append(df)
            print(f"✅ {len(df)} teams")
        else:
            print("⚠️ No data")
        
        time.sleep(1.5)
    
    # Save current season
    if current_data:
        current_df = pd.concat(current_data, ignore_index=True)
        current_df = current_df.sort_values(['league', 'pts'], ascending=[True, False])
        current_df['position'] = current_df.groupby('league').cumcount() + 1
        current_df['scrape_timestamp'] = datetime.now().isoformat()
        
        current_df.to_csv(f'understat_teams_aggregated_{current_season}_latest.csv', index=False)
        current_df.to_parquet(f'understat_teams_aggregated_{current_season}_latest.parquet', index=False)
        print(f"\n✅ Current season saved: {len(current_df)} teams")
    
    # Combined dataset
    if historical_data and current_data:
        combined_df = pd.concat([historical_df, current_df], ignore_index=True)
        combined_df = combined_df.sort_values(['year', 'league', 'pts'], ascending=[True, True, False])
        
        combined_df.to_csv('understat_teams_aggregated_2014_td.csv', index=False)
        combined_df.to_parquet('understat_teams_aggregated_2014_td.parquet', index=False)
        print(f"\n✅ Combined dataset saved: {len(combined_df)} total team-seasons")
    
    print("\n" + "=" * 60)
    print("📊 TEAM DATA SCRAPING COMPLETE!")
    
    # Summary statistics
    if combined_df is not None:
        print(f"\nSummary:")
        print(f"  Total records: {len(combined_df)}")
        print(f"  Seasons: {combined_df['year'].nunique()}")
        print(f"  Leagues: {combined_df['league'].nunique()}")
        print(f"  Unique teams: {combined_df['team_name'].nunique()}")

if __name__ == "__main__":
    scrape_all_team_data()

⚽ UNDERSTAT TEAM DATA SCRAPER

📚 Scraping historical data (2014-2024)...

 Season 2014:
  EPL... ✅ 20 teams
  La_Liga... ✅ 20 teams
  Bundesliga... ✅ 18 teams
  Serie_A... ✅ 20 teams
  Ligue_1... ✅ 20 teams
  RFPL... ✅ 16 teams

 Season 2015:
  EPL... ✅ 20 teams
  La_Liga... ✅ 20 teams
  Bundesliga... ✅ 18 teams
  Serie_A... ✅ 20 teams
  Ligue_1... ✅ 20 teams
  RFPL... ✅ 16 teams

 Season 2016:
  EPL... ✅ 20 teams
  La_Liga... ✅ 20 teams
  Bundesliga... ✅ 18 teams
  Serie_A... ✅ 20 teams
  Ligue_1... ✅ 20 teams
  RFPL... ✅ 16 teams

 Season 2017:
  EPL... ✅ 20 teams
  La_Liga... ✅ 20 teams
  Bundesliga... ✅ 18 teams
  Serie_A... ✅ 20 teams
  Ligue_1... ✅ 20 teams
  RFPL... ✅ 16 teams

 Season 2018:
  EPL... ✅ 20 teams
  La_Liga... ✅ 20 teams
  Bundesliga... ✅ 18 teams
  Serie_A... ✅ 20 teams
  Ligue_1... ✅ 20 teams
  RFPL... ✅ 16 teams

 Season 2019:
  EPL... ✅ 20 teams
  La_Liga... ✅ 20 teams
  Bundesliga... ✅ 18 teams
  Serie_A... ✅ 20 teams
  Ligue_1... ✅ 20 teams
  RFPL... ✅ 16 tea

In [3]:
# Read the aggregated Understat teams CSV into a DataFrame
file_path = "/Users/rchaudhary/understat_teams_data/understat_teams_aggregated_2014_td.csv"

if not os.path.exists(file_path):
    raise FileNotFoundError(f"File not found: {file_path}")

try:
    df_understat = pd.read_csv(file_path, low_memory=False)
    print(f"Loaded {len(df_understat):,} rows and {len(df_understat.columns):,} columns from {file_path}")
    display(df_understat.head())
except Exception as e:
    print(f"Error reading CSV: {e}")

Loaded 1,362 rows and 27 columns from /Users/rchaudhary/understat_teams_data/understat_teams_aggregated_2014_td.csv


,team_id,team_name,league,year,season,matches,wins,draws,losses,scored,...,pts,xpts,ppda_coef,oppda_coef,npxGD,xG_diff,xGA_diff,xpts_diff,position,scrape_timestamp
0,117,Bayern Munich,Bundesliga,2014,2014/15,34,25,4,5,80,...,79,73.6116,5.654984,17.785539,41.236085,-13.794255,3.938729,-5.3884,1,2025-09-01T17:43:15.144962
1,131,Wolfsburg,Bundesliga,2014,2014/15,34,20,9,5,72,...,69,59.9509,8.428571,9.662175,18.897371,-13.652269,0.822021,-9.0491,2,2025-09-01T17:43:15.144962
2,130,Borussia M.Gladbach,Bundesliga,2014,2014/15,34,19,9,6,53,...,66,57.0078,11.413437,11.929325,16.156039,-1.258428,10.343299,-8.9922,3,2025-09-01T17:43:15.144962
3,119,Bayer Leverkusen,Bundesliga,2014,2014/15,34,17,10,7,62,...,61,55.3396,5.500816,6.611518,18.224194,-12.844301,-2.279614,-5.6604,4,2025-09-01T17:43:15.144962
4,121,Augsburg,Bundesliga,2014,2014/15,34,15,4,15,43,...,49,45.4960,7.064975,9.394883,-5.908343,0.911698,3.659608,-3.5040,5,2025-09-01T17:43:15.144962
